# Multivariate plotting

<table>
<tr>
<td><img src="https://i.imgur.com/gJ65O47.png" width="350px"/></td>
<td><img src="https://i.imgur.com/3qEqPoD.png" width="350px"/></td>
<td><img src="https://i.imgur.com/1fmV4M2.png" width="350px"/></td>
<td><img src="https://i.imgur.com/H20s88a.png" width="350px"/></td>
</tr>
<tr>
<td style="font-weight:bold; font-size:16px;">Multivariate Scatter Plot</td>
<td style="font-weight:bold; font-size:16px;">Grouped Box Plot</td>
<td style="font-weight:bold; font-size:16px;">Heatmap</td>
<td style="font-weight:bold; font-size:16px;">Parallel Coordinates</td>
</tr>
<tr>
<td>df.plot.scatter()</td>
<td>df.plot.box()</td>
<td>sns.heatmap</td>
<td>pd.plotting.parallel_coordinates</td>
</tr>
<!--
<tr>
<td>Good for interval and some nominal categorical data.</td>
<td>Good for interval and some nominal categorical data.</td>
<td>Good for nominal and ordinal categorical data.</td>
<td>Good for ordinal categorical and interval data.</td>
</tr>
-->
</table>


이 튜토리얼의 대부분은 데이터를 하나의 (단일 변량) 또는 2 개 (2 변수) 차원으로 플롯팅했습니다. 이전 섹션에서 우리는 데이터를 ""gridding out"하여 작동하는 다변수 플로팅 방법인  faceting을 탐구했습니다 .

이 섹션에서는 다 변수 플로팅에 대해 자세히 살펴 보겠습니다. 먼저 '진정한'다 변수 차트를 살펴 보겠습니다. 그런 다음 요약을 사용하여 같은 일을하는 일부 플롯을 검토합니다.

In [ ]:
import pandas as pd
pd.set_option('max_columns', None)
df = pd.read_csv("../input/fifa-18-demo-player-dataset/CompleteDataset.csv", index_col=0)

import re
import numpy as np

footballers = df.copy()
footballers['Unit'] = df['Value'].str[-1]
footballers['Value (M)'] = np.where(footballers['Unit'] == '0', 0, 
                                    footballers['Value'].str[1:-1].replace(r'[a-zA-Z]',''))
footballers['Value (M)'] = footballers['Value (M)'].astype(float)
footballers['Value (M)'] = np.where(footballers['Unit'] == 'M', 
                                    footballers['Value (M)'], 
                                    footballers['Value (M)']/1000)
footballers = footballers.assign(Value=footballers['Value (M)'],
                                 Position=footballers['Preferred Positions'].str.split().str[0])

(참고 : 위의 첫 번째 코드 셀에는 일부 데이터 전처리가 포함되어 있습니다.이 것은 관련이 없으므로 기본적으로 숨겨져 있습니다.)

In [ ]:
footballers.head()

## Adding more visual variables

변수를 많이 사용하는 가장 확실한 방법은 우리가 지금까지 사용해온 시각화를  [visual variables](http://www.infovis-wiki.net/index.php?title=Visual_Variables)와 함께 확대해 보는 것입니다. ** 시각적 변수 **는 두 개의 데이터 요소를 서로 지각 적으로 구별하는 데 사용할 수 있는 시각적인 치수 또는 표식을 나타냅니다. 예를 들면 크기, 색상, 모양, 1, 2 ,3 차원 위치 등이 있습니다.

"Good" 다변수 데이터 표시는 이러한 매개 변수를 효율적으로 쉽게 해석 할 수있 는 방법입니다.

### Multivariate scatter plots

산점도를 통해서 스트라이커, 우익수, 좌익수 중 어떤 공격수가 가장 보수를 많이 받는지 알아 볼 것입니다.

In [ ]:
import seaborn as sns

sns.lmplot(x='Value', y='Overall', hue='Position', 
           data=footballers.loc[footballers['Position'].isin(['ST', 'RW', 'LW'])], 
           fit_reg=False)


이 산점도는 세 가지 시각적 변수를 사용합니다. 수평 위치 (x 값)는 플레이어의 'Value'를 추적합니다 (지불 방법 등). 수직 위치 (y 값)는 모든 속성에 걸쳐 플레이어의 `Overall` 점수를 추적합니다. 그리고 색상 (`hue` 매개 변수)은 플레이어가 가리키는 3 가지 관심 분야 중 어느 것이 있는지 추적합니다.

이 차트의 새 변수는 ** color **입니다. Color는 심미적으로 즐거운 시각적 정보를 제공하지만 사용하기 까다롭습니다. 이 산점도를 살펴보면 이전 섹션에서 보았던 동일한 overplotting 문제를 볼 수 있습니다. 그러나 16 진수 플롯을 사용하는 것과 같은 더 쉬운 솔루션은 더 이상 존재하지 않습니다. 왜냐하면 색상은 그 설정에서 의미가 없기 때문입니다.

다른 시각적 변수의 예로는 ** shape **이 있습니다. Shape controls 마커의 shape을 나타냅니다. :

In [ ]:
sns.lmplot(x='Value', y='Overall', markers=['o', 'x', '*'], hue='Position',
           data=footballers.loc[footballers['Position'].isin(['ST', 'RW', 'LW'])],
           fit_reg=False
          )

`seaborn`은 어떤 종류의 시각적 변수를 사용해야하는지에 대해서,  shape 옵션을 제공하지 않습니다. 그 이유는 단순한 shape이 멋지지만 구별력 면에서 색상에 비해 지각하기 어렵기 때문입니다.

### Grouped box plot

또Grouped box plot을 통해서 "스트라이커가 골키퍼보다 "공격성"에서 더 높은 점수를 얻나요?"라는 질문에 답해 보겠습니다.

In [ ]:
f = (footballers
         .loc[footballers['Position'].isin(['ST', 'GK'])]
         .loc[:, ['Value', 'Overall', 'Aggression', 'Position']]
    )
f = f[f["Overall"] >= 80]
f = f[f["Overall"] < 85]
f['Aggression'] = f['Aggression'].astype(float)

sns.boxplot(x="Overall", y="Aggression", hue='Position', data=f)


보시다시피,이 그래프는 우리의 데이터 세트 내에서 골키퍼 (적어도 80 점과 85 점 사이의 전반적인 점수를 가진 사람들)가 스트라이커보다 공격 점수가 훨씬 더 낮음을 결정적으로 보여줍니다.

이 그림에서 가로 축은 '전체'점수를 인코딩하고, 세로 축은 '침략'점수를 인코딩하고 그룹은 '위치'를 인코딩합니다.

그룹화는 의사 소통이 가능한 시각적 변수입니다.이 차트는 매우 쉽게 해석 할 수 있습니다. 그러나 카디널리티가 매우 낮습니다. 그룹을 소수의 범주 형 값 이상으로 사용하는 것은 매우 어렵습니다. 이 플롯에서 우리는 단지 2 명의 플레이어 포지션과 5 개의 전반적인 플레이어 점수를 선택했는데도 시각화 결과는  다소 혼잡합니다. 전반적으로 그룹화는 할 수있는 것과 수행 할 수없는 것이 있다는 점에서 faceting과 매우 유사합니다.

## 요약

해석 가능성을 해치지 않으면 서 플롯에 충분한 치수를 집어 넣기가 어렵습니다. 매우 복잡한 구획은 당연히 해석하기가 어렵습니다. 따라서 고도로 다변수는 사용하기가 어려울 수 있습니다.

이 문제를 극복하면서 많은 데이터 세트 기능을 계획하는 또 다른 방법은 ** summarization **을 사용하는 것입니다. 요약은 이전 변수에 제공된 정보를 혼합하고 일치시켜 새로운 변수를 만들고 추가하는 것입니다.

요약은 잠재적으로 매우 복잡한 관계를 단순한 것으로 나타낼 수  있기 때문에 데이터 시각화에서 유용한 기술입니다.

### Heatmap

 correlation plot을 통해서 데이터 세트의 모든 값의 쌍 간의 상관 관계를 측정하고 결과를 색상으로 표시합니다.

In [ ]:
f = (
    footballers.loc[:, ['Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control']]
        .applymap(lambda v: int(v) if str.isdecimal(v) else np.nan)
        .dropna()
).corr()

sns.heatmap(f, annot=True)

이 플롯의 각 셀은 두 변수의 교차점입니다. 

색상과 레이블은 함께 두 변수 사이의 상관 관계를 나타냅니다 (두 변수가 동시에 증가 또는 감소 할 가능성). 

예를 들어,이 데이터 세트에서 Agility와 Acceleration은 높은 상관 관계가 있는 반면 Aggression과 Balance은 관련이 거의 없습니다.

상관 관계 플롯은 특정 종류의 히트 맵입니다. **히트 맵**은 데이터 세트에서 선택한 모든 변수 쌍에 대한 특정 사실 (이 경우 상관 관계)을 매핑합니다.


### Parallel Coordinates

**parallel coordinates plot** 은 여러 변수에서 데이터를 시각화하는 또 다른 방법을 제공합니다.

In [ ]:
from pandas.plotting import parallel_coordinates

f = (
    footballers.iloc[:, 12:17]
        .loc[footballers['Position'].isin(['ST', 'GK'])]
        .applymap(lambda v: int(v) if str.isdecimal(v) else np.nan)
        .dropna()
)
f['Position'] = footballers['Position']
f = f.sample(200)

parallel_coordinates(f, 'Position')


위의 시각화에서 관심 있는 5 가지 변수를 통해 200 명의 골키퍼 (짙은 녹색)와 스트라이커 (밝은 녹색)의 샘플을 작성했습니다.

평행 좌표 플롯(Parallel coordinates plot)은 다른 클래스가 데이터에서 얼마나 구별 가능한지를 결정할 때 유용합니다. 변수를 위에서부터 아래로 표준화합니다 ... 이 경우, 스트라이커는 우리가 선택한 모든 변수에서 거의 균일하게 높은 평가를 받습니다. 즉, 이 두 클래스의 플레이어는 구분하기가 쉽습니다.

## 연습문제

In [ ]:
pokemon = pd.read_csv("../input/pokemon/Pokemon.csv", index_col=0)
pokemon.head()

다음 질문에 답하십시오. 답변을 보려면 아래 셀의 '출력'버튼을 클릭하십시오.

1. 다변수 데이터 시각화를 생성하는 세 가지 기술은 무엇입니까?
2. 시각적 변수의 세 가지 예를 나열하십시오.
3. 데이터 시각화의 요약하면 무엇입니까?


이 튜토리얼에서 다뤘던 세 가지 기술은 faceting, 더 많은 시각적 변수 사용 및 요약입니다.

시각적 변수의 예로는 모양, 색, 크기, x 위치, y 위치 및 그룹을 들 수 있습니다. 그러나 가능한 많은 것들이 더 있습니다!

데이터 시각화에서는 복잡한 데이터를 보다 간단하고 플롯하기 쉬운 지표로 요약  압축합니다.

In [ ]:
from IPython.display import HTML
HTML("""
<ol>
<li>The three techniques we have covered in this tutorial are faceting, using more visual variables, and summarization.</li>
<br/>
<li>Some examples of visual variables are shape, color, size, x-position, y-position, and grouping. However there are many more that are possible!</li>
<br/>
<li>In data visualization, summarization works by compressing complex data into simpler, easier-to-plot indicators.</li>
</ol>
""")

**연습

In [ ]:
import seaborn as sns

sns.lmplot(x='Attack', y='Defense', hue='Legendary', 
           markers=['x', 'o'],
           fit_reg=False, data=pokemon)

In [ ]:
sns.boxplot(x="Generation", y="Total", hue='Legendary', data=pokemon)

In [ ]:
sns.heatmap(
    pokemon.loc[:, ['HP', 'Attack', 'Sp. Atk', 'Defense', 'Sp. Def', 'Speed']].corr(),
    annot=True
)

In [ ]:
import pandas as pd
from pandas.plotting import parallel_coordinates

p = (pokemon[(pokemon['Type 1'].isin(["Psychic", "Fighting"]))]
         .loc[:, ['Type 1', 'Attack', 'Sp. Atk', 'Defense', 'Sp. Def']]
    )

parallel_coordinates(p, 'Type 1')

# 결론

이 튜토리얼에서  2 개의 이상의  다변수 데이터 시각화 기술로 진입했습니다.

 첫 번째 기법은 더 많은 시각적 변수를 추가하는 것으로 더 복잡하지만 잠재적으로보다 자세한 플롯을 생성합니다. 두 번째 기법인 요약은 가변적인 정보를 요약 통계로 압축하여 표현력을 희생 시키더라도 간단한 출력을 제공합니다.

이 튜토리얼에서 다룰 3 가지 다변량 기법은 Faceting, visual variables및 summarization 을 추가했습니다.

나머지 내용은 선택 사항입니다. 다음 섹션에서는이 라이브러리를 기반으로하는 매우 인기있는 인터랙티브 시각화 라이브러리`plotly`를 사용하는 방법을 배웁니다.

[Click here to go to the next section, "Introduction to plotly"](https://www.kaggle.com/residentmario/introduction-to-plotly-optional/).